In [1]:
from AREgym import AREEnv

import gym


from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env

from stable_baselines3.ppo import MlpPolicy

from torch.utils.tensorboard import SummaryWriter
# env = env = gym.make('Pendulum-v0')
# env = AREEnv(250,10)
import numpy as np

# env = AREEnv(250, 10, num_laser_scan=10, heuristic_dist=10, save_map=True)
# # print(env.reset())
# # env.render()

# for i in range(2000):
#     env.step((np.random.rand() - 0.5) * 2)
# # env.laserscan
# print(env.step(1))
# env.render()



In [2]:
# env = AREEnv(250, 10, num_laser_scan=10, save_map=False, max_steps=128)
# env = AREEnv(500, 20, save_map=True)
# check_env(env)


In [2]:
from stable_baselines3.common.env_util import make_vec_env

env_kwargs = dict(grid=250, step_distance=10, num_laser_scan=10, save_map=False, max_steps=256)

env = make_vec_env(AREEnv, n_envs=4, env_kwargs=env_kwargs)


In [3]:
# policy_kwargs = dict(net_arch=dict(pi=[256, 128, 64], vf=[256, 128, 64]))
# import torch as th
policy_kwargs = dict(net_arch=[dict(pi=[128, 64], vf=[128, 64])])

In [5]:
model = PPO("MlpPolicy",
            env,
            verbose=0,
            n_steps=256,
            tensorboard_log='sb_runs/256',
            # batch_size=128,
            batch_size=1024,
            policy_kwargs=policy_kwargs,
            seed=1
            )
# model = PPO("MlpPolicy", env, verbose=1, gamma=0.9)
model.learn(total_timesteps=2000000)
model.save("stable_baselines3_ppo")

/home/sheng/work/SARE-planner/AREgym.py:264: RuntimeWarning: invalid value encountered in multiply
  map_ = np.nan_to_num(((self.global_map - 1) * -np.inf).astype(np.float32), nan=1, posinf=np.inf)


In [9]:
model.save("stable_baselines3_ppo_256_2m")

In [4]:
model = PPO.load("stable_baselines3_ppo_256_2m")

/home/qunshen/anaconda3/envs/sare/lib/python3.6/site-packages/cloudpickle/cloudpickle.py:704: UserWarning: A pickle file created using an old (<=1.4.1) version of cloudpicke is currently being loaded. This is not supported by cloudpickle and will break in cloudpickle 1.7
  "will break in cloudpickle 1.7", category=UserWarning


In [7]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential(
      (0): Linear(in_features=20, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=20, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [5]:
import imageio
env = AREEnv(250, 10, num_laser_scan=10, save_map=True, max_steps=256)

obs = env.reset()

img1_lst, img2_lst, img3_lst = [], [], []
for i in range(256):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    if dones:
        print('terminate')
        break
    # print(i)
    img1, img2, img3 = env.render()
    img1_lst.append(img1)
    img2_lst.append(img2)
    img3_lst.append(img3)

imageio.mimsave("global_map.gif", [np.array(img) for i, img in enumerate(img1_lst) if i%2 == 0], fps=29)
imageio.mimsave("world_map.gif", [np.array(img) for i, img in enumerate(img2_lst) if i%2 == 0], fps=29)
imageio.mimsave("current_state.gif", [np.array(img) for i, img in enumerate(img3_lst) if i%2 == 0], fps=29)

/home/qunshen/SARE-planner/AREgym.py:264: RuntimeWarning: invalid value encountered in multiply
  map_ = np.nan_to_num(((self.global_map - 1) * -np.inf).astype(np.float32), nan=1, posinf=np.inf)


terminate


In [ ]:
print(env.reset())
env.render()

In [ ]:
print(obs)

In [ ]:
# import gym

# from stable_baselines3 import PPO
# from stable_baselines3.common.env_util import make_vec_env

# # Parallel environments
# env = make_vec_env("CartPole-v1", n_envs=4)

# model = PPO("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=25000)
# model.save("ppo_cartpole")

# del model # remove to demonstrate saving and loading

# model = PPO.load("ppo_cartpole")

# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     env.render()